# 构建卷积神经网络
- 卷积神经网络的输入层与传统神经网络有些区别，需要重新设计，但是训练模块是基本一致的

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets,transforms
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

e:\download\python\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 首先读取数据
- 分别构建训练接和检测集（验证集）
- DataLoader来迭代取数据

In [2]:
# 定义超参数
input_size = 28 # 图像的总尺寸是28*28
num_classes = 10
num_epochs = 3
batch_size = 64


In [23]:
# 训练集
train_dataset = datasets.MNIST(root='./data',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

# 测试集
test_dataset = datasets.MNIST(root='./data',
                              train = False,
                               transform=transforms.ToTensor())

In [22]:
train_dataset

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [4]:
# 构建batch数据
from random import shuffle


train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

# 卷积网络模块构建
- 一般卷积层，relu层是一起的，两次卷积一次池化
- 注意卷积最后结果还是一个特征图，需要把图转化为向量再做分类或者回归任务

In [5]:
from turtle import forward


class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.conv1 = nn.Sequential(  # 输入大小为（1,28,28）
            nn.Conv2d(
                in_channels=1,          # 灰度图
                out_channels=16,        # 要得到多少个特征图
                kernel_size=5,          # 卷积核大小
                stride=1,               # 步长
                padding=2               # 如果希望卷积后的特征图的大小跟原来一样,需要设置padding=(kernel_size-1)/2 if stride=1
            ),                          # 此时输出的特征图为（16，28，28） 
            nn.ReLU(),                  # ReLU层
            nn.MaxPool2d(kernel_size=2),# 进行池化操作（2x2），输出结果为(16,14,14)
        )
        self.conv2 = nn.Sequential(     # 下个一个卷积输入为（16，14，14）
            nn.Conv2d(16,32,5,1,2),     # 输出（32，14，14）
            nn.ReLU(),                  # ReLU层
            nn.MaxPool2d(2)             # 输出（32，7，7）
        )
        self.out = nn.Linear(32*7*7,10) # 全连接层得到的结果
    def forward(self,x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0),-1)        # flatten操作，结果是(batch_size,32x7x7)
        output = self.out(x)
        return output

In [6]:
# 这将准去率作为最终的评估标准
def accuracy(predictions,labels):
    pred = torch.max(predictions.data,1)[1]
    rights = pred.eq(labels.data.view_as(pred)).sum()
    return rights,len(labels)

In [9]:
# 训练网络模型
# 实例化
net = CNN()
# 损失函数
criterion = nn.CrossEntropyLoss()
# 优化器
optimizer = optim.Adam(net.parameters(),lr=0.01) # 定义优化器

# 开始训练循环
for epoch in range(num_epochs):
    train_rights = []
    for batch_idx,(data,target) in enumerate(train_loader): # 针对容器中的每一个批进行循环
        net.train()
        output = net(data)
        loss = criterion(output,target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        right = accuracy(output,target)
        train_rights.append(right)
        
        if batch_idx % 100 == 0:
            net.eval()
            val_rights = []
            for (data,target) in test_loader:
                output = net(data)
                right = accuracy(output,target)
                val_rights.append(right)
                
            # 准确率计算
            train_r = (sum([tup[0] for tup in train_rights]),sum([tup[1] for tup in train_rights]))
            val_r = (sum([tup[0] for tup in val_rights]),sum([tup[1] for tup in val_rights]))
            
            print("当前epoch:{} [{}/{} ({:.0f}%)]\t损失:{:6f}\t训练集准确率:{:.2f}%\t检测集准确率:{:.2f}%".format(
                epoch,batch_idx*batch_size,len(train_loader.dataset),
                100.*batch_idx/len(train_loader),
                loss.data,
                100.*train_r[0].numpy()/train_r[1],
                100.*val_r[0]/val_r[1]
            ))
        

当前epoch:0 [0/60000 (0%)]	损失:2.314794	训练集准确率:9.38%	检测集准确率:18.42%
当前epoch:0 [6400/60000 (11%)]	损失:0.173669	训练集准确率:85.32%	检测集准确率:96.22%
当前epoch:0 [12800/60000 (21%)]	损失:0.062894	训练集准确率:90.59%	检测集准确率:96.90%
当前epoch:0 [19200/60000 (32%)]	损失:0.062483	训练集准确率:92.68%	检测集准确率:96.75%
当前epoch:0 [25600/60000 (43%)]	损失:0.028073	训练集准确率:93.86%	检测集准确率:97.67%
当前epoch:0 [32000/60000 (53%)]	损失:0.081514	训练集准确率:94.55%	检测集准确率:97.38%
当前epoch:0 [38400/60000 (64%)]	损失:0.030849	训练集准确率:95.00%	检测集准确率:97.95%
当前epoch:0 [44800/60000 (75%)]	损失:0.019593	训练集准确率:95.39%	检测集准确率:97.92%
当前epoch:0 [51200/60000 (85%)]	损失:0.032896	训练集准确率:95.66%	检测集准确率:98.00%
当前epoch:0 [57600/60000 (96%)]	损失:0.054455	训练集准确率:95.85%	检测集准确率:97.74%
当前epoch:1 [0/60000 (0%)]	损失:0.219851	训练集准确率:96.88%	检测集准确率:98.43%
当前epoch:1 [6400/60000 (11%)]	损失:0.056785	训练集准确率:97.94%	检测集准确率:98.54%
当前epoch:1 [12800/60000 (21%)]	损失:0.067520	训练集准确率:98.14%	检测集准确率:97.93%
当前epoch:1 [19200/60000 (32%)]	损失:0.051063	训练集准确率:98.12%	检测集准确率:98.11%
当前epoch:1 [25600/60000 (43%)]	损失:

In [7]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

1.12.1+cu113
True


In [18]:
from turtle import forward
# 这将准去率作为最终的评估标准
def accuracy(predictions,labels):
    pred = torch.max(predictions.data.cuda(),1)[1]
    rights = pred.eq(labels.cuda().data.view_as(pred.cuda())).sum()
    return rights,len(labels)

class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.conv1 = nn.Sequential(  # 输入大小为（1,28,28）
            nn.Conv2d(
                in_channels=1,          # 灰度图
                out_channels=16,        # 要得到多少个特征图
                kernel_size=5,          # 卷积核大小
                stride=1,               # 步长
                padding=2               # 如果希望卷积后的特征图的大小跟原来一样,需要设置padding=(kernel_size-1)/2 if stride=1
            ),                          # 此时输出的特征图为（16，28，28） 
            nn.ReLU(),                  # ReLU层
            nn.MaxPool2d(kernel_size=2),# 进行池化操作（2x2），输出结果为(16,14,14)
        )
        self.conv2 = nn.Sequential(     # 下个一个卷积输入为（16，14，14）
            nn.Conv2d(16,32,5,1,2),     # 输出（32，14，14）
            nn.ReLU(),                  # ReLU层
            nn.MaxPool2d(2)             # 输出（32，7，7）
        )
        self.out = nn.Linear(32*7*7,10) # 全连接层得到的结果
    def forward(self,x):
        x = self.conv1(x.cuda())
        x = self.conv2(x.cuda())
        x = x.view(x.size(0),-1)        # flatten操作，结果是(batch_size,32x7x7)
        output = self.out(x)
        return output

In [ ]:
# 构建batch数据
from random import shuffle


train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

In [24]:
# 训练网络模型
# 实例化
net = CNN()
net.to("cuda")
# 损失函数
criterion = nn.CrossEntropyLoss()
# 优化器
optimizer = optim.Adam(net.parameters(),lr=0.01) # 定义优化器

# 开始训练循环
for epoch in range(num_epochs):
    train_rights = []
    for batch_idx,(data,target) in enumerate(train_loader): # 针对容器中的每一个批进行循环
        net.train()
        output = net(data)
        loss = criterion(output.cuda(),target.cuda())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        right = accuracy(output,target)
        train_rights.append(right)
        
        if batch_idx % 100 == 0:
            net.eval()
            val_rights = []
            for (data,target) in test_loader:
                output = net(data)
                right = accuracy(output,target)
                val_rights.append(right)
                
            # 准确率计算
            train_r = (sum([tup[0] for tup in train_rights]),sum([tup[1] for tup in train_rights]))
            val_r = (sum([tup[0] for tup in val_rights]),sum([tup[1] for tup in val_rights]))
            
            print("当前epoch:{} [{}/{} ({:.0f}%)]\t损失:{:6f}\t训练集准确率:{:.2f}%\t检测集准确率:{:.2f}%".format(
                epoch,batch_idx*batch_size,len(train_loader.dataset),
                100.*batch_idx/len(train_loader),
                loss.data,
                100.*train_r[0]/train_r[1],
                100.*val_r[0]/val_r[1]
            ))

当前epoch:0 [0/60000 (0%)]	损失:2.300650	训练集准确率:6.25%	检测集准确率:9.86%
当前epoch:0 [6400/60000 (11%)]	损失:0.146061	训练集准确率:80.57%	检测集准确率:94.02%
当前epoch:0 [12800/60000 (21%)]	损失:0.158085	训练集准确率:88.00%	检测集准确率:96.00%
当前epoch:0 [19200/60000 (32%)]	损失:0.029171	训练集准确率:90.86%	检测集准确率:96.48%
当前epoch:0 [25600/60000 (43%)]	损失:0.059782	训练集准确率:92.47%	检测集准确率:96.72%
当前epoch:0 [32000/60000 (53%)]	损失:0.150853	训练集准确率:93.36%	检测集准确率:97.76%
当前epoch:0 [38400/60000 (64%)]	损失:0.125140	训练集准确率:93.91%	检测集准确率:97.20%
当前epoch:0 [44800/60000 (75%)]	损失:0.026381	训练集准确率:94.43%	检测集准确率:97.66%
当前epoch:0 [51200/60000 (85%)]	损失:0.043444	训练集准确率:94.78%	检测集准确率:97.98%
当前epoch:0 [57600/60000 (96%)]	损失:0.009474	训练集准确率:95.05%	检测集准确率:97.07%
当前epoch:1 [0/60000 (0%)]	损失:0.115455	训练集准确率:95.31%	检测集准确率:97.89%
当前epoch:1 [6400/60000 (11%)]	损失:0.230221	训练集准确率:97.34%	检测集准确率:97.54%
当前epoch:1 [12800/60000 (21%)]	损失:0.037741	训练集准确率:97.54%	检测集准确率:97.85%
当前epoch:1 [19200/60000 (32%)]	损失:0.185074	训练集准确率:97.61%	检测集准确率:97.91%
当前epoch:1 [25600/60000 (43%)]	损失:0